In [ ]:
sc = spark.sparkContext
oneSysLog = sc.textFile("file:/var/log/system.log")
allSysLogs = sc.textFile("file:/var/log/system.log*")
allLogs = sc.textFile("file:/var/log/*.log*")

In [ ]:
print oneSysLog.count()
print allSysLogs.count()
print allLogs.count()

print oneSysLog.collect()

In [ ]:
from pyspark.sql import Row
logsRDD = allSysLogs.map(lambda logRow: Row(log=logRow))
logsDF = spark.createDataFrame(logsRDD)
logsDF.createOrReplaceTempView("logs")

logsDF.printSchema()
spark.sql("select log from logs limit ").show()

In [ ]:
spark.sql("select log from logs limit 4").show()

In [ ]:
peopleDF = spark.read.json("examples/src/main/resources/people.json")
peopleDF.write.parquet("people.parquet")

parquetFile = spark.read.parquet("people.parquet")
parquetFile.printSchema()
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")
teenagers.show()


In [ ]:
import time
start_time = time.time()

sqlUrl = "jdbc:mysql://127.0.0.1:32768?user=root&password=root"
df = spark.read.jdbc(url=sqlUrl, table="kudo.core_order_items", 
                     properties={"driver": 'com.mysql.jdbc.Driver'})

df1 = spark.read.jdbc(url=sqlUrl, table="kudo.core_vendors", 
                     properties={"driver": 'com.mysql.jdbc.Driver'})
df = df.alias("a")
df1 = df1.alias("b")
df2= df.join(df1, df1.id==df.vendor_id)
df2.printSchema()
df2.select("a.id").show()
print("--- %s seconds ---" % (time.time() - start_time))




In [ ]:
df = df.alias("a")
df1 = df1.alias("b")
peopleDF = spark.read.json("examples/src/main/resources/people-1.json")
peopleDF = peopleDF.alias("c")
start_time = time.time()
df2= df.join(df1, df1.id==df.vendor_id)
df3= df1.join(peopleDF, peopleDF.id==df1.id)
df3.show()
print("--- %s seconds ---" % (time.time() - start_time))
df2.select("a.id").show()


